# Example notebook for loading TOFWERK HD5 dataset

Load the FIB image and peak data

In [ ]:
import twtof

filename = "./data/20230722_magnetotactic_HO027_1_Au_pos_spot2_run1_30kV_50pA.h5"
filename = "./data/20241017_Fluorine_HO_134_1_spot8_run1_30kV_50pA.h5"
fib, mass, peak = twtof.imread(filename)

print(f"Loaded FIB image data with shape {fib.shape}")
print(f"Loaded mass information with shape {mass.shape}")
print(f"Loaded peak data with shape {peak.shape}")

In [ ]:
d = twtof.as_dict(filename)

In [ ]:
d

In [ ]:
# twtof.print_content(filename)

Interactive display of the image and the associated spectrum

In [ ]:
%matplotlib ipympl
from ipywidgets import interact
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
axfib = ax[0].imshow(fib[0].squeeze())
crosshair = ax[0].plot(0, 0, "w+", ms=50)
axpeak = ax[1].semilogy(1e-6+peak[0, 0, 0].ravel())
sline = ax[1].semilogy((0, 0), (0, 2), "r-")
axmap = ax[2].imshow(peak[0, :, :, 0].squeeze())
ax[1].set(xlabel="mass", ylabel="count")
ax[1].set_box_aspect(1)
ax[0].set_axis_off()
ax[2].set_axis_off()


@interact(
    x=(0, fib.shape[2] - 1, 1),
    y=(0, fib.shape[1] - 1, 1),
    z=(0, fib.shape[0] - 1, 1),
    c=(0, peak.shape[3] - 1, 1),
    cmap=["viridis", "jet", "gray"],
)
def display_twotof(x, y, z, c, cmap):
    axfib.set(data=fib[z], cmap=cmap)
    crosshair[0].set(data=([x], [y]))
    ax[0].set_title(f"FIB Image\n({x},{y},{z})")
    d = int(fib.shape[1] / peak.shape[1])
    axpeak[0].set(data=(mass, peak[z, y // d, x // d].ravel()))
    sline[0].set(data=((c, c), (0, 2)))
    ax[1].set_title(f"mass {c}")
    axmap.set(data=peak[z, :, :, c].squeeze(), cmap=cmap)
    ax[2].set_title("Peak data")

Load the peak table

In [ ]:
with twtof.TofH5Reader(filename) as f:
    peak_df = f.load_peak_table()
peak_df

Load all the data in a dictionnary

In [ ]:
data = twtof.as_dict(filename)
data["Acquisition log"]

# Export FIB image and peak to TIF

In [ ]:
import twtof
import tifffile
import numpy as np

filename = "./data/20230722_magnetotactic_HO027_1_Au_pos_spot2_run1_30kV_50pA.h5"

fib, mass, peak = twtof.imread(filename)

tifffile.imwrite(
    "mass.tif",
    np.moveaxis(peak, [0, 3, 1, 2], [0, 1, 2, 3]),
    imagej=True,
    metadata={"axes": "ZCYX"},
)

tifffile.imwrite(
    "fib.tif", fib.astype(np.uint16), imagej=True, metadata={"axes": "ZYX"}
)

# Average spectra in ROI

Select ROI in the FIB image and display the corresponding averaged spectra.


In [ ]:
import matplotlib.pyplot as plt
from mpl_image_segmenter import ImageSegmenter
import ipywidgets as widgets

fig, ax = plt.subplots(1, 2)

# Define the numer of regions
N_ROI = 3

# Create the widgets
slice_selector = widgets.IntSlider(
    value=0, min=0, max=fib.shape[0] - 1, description="Slice"
)
class_selector = widgets.Dropdown(options=list(range(1, N_ROI + 1)), description="ROI")
erasing_button = widgets.Checkbox(value=False, description="Erasing")
refresh_btn = widgets.Button(value=False, description="Refresh")


# Update function
def update(change):
    # update the active ROI/class
    multi_class_segmenter.current_class = class_selector.value
    # toggle to erase mode
    multi_class_segmenter.erasing = erasing_button.value
    # set the image plane in the left panel
    multi_class_segmenter._displayed.set_data(fib[slice_selector.value])

    # retreive the current mask and downscale it
    d = fib.shape[1] // peak.shape[1]
    lbl = multi_class_segmenter.mask[::d, ::d].astype(int)
    # for each label
    for c in np.unique(lbl)[1:]:
        # draw the averaged spectrum in the region
        f = peak[slice_selector.value, lbl == c, :].mean(axis=0)
        axpeak[c - 1][0].set_data(mass, 1e-3 + f)
    refresh_btn.value = False


# Add callbacks
erasing_button.observe(update, names="value")
class_selector.observe(update, names="value")
slice_selector.observe(update, names="value")
refresh_btn.on_click(update)


ax[0].set_axis_off()
axpeak = [ax[1].semilogy([0, mass.max()], [1e-3, 1e-3]) for k in range(N_ROI)]
ax[1].set_box_aspect(1)
ax[1].set(xlim=(0, mass.max()), ylim=(1e-3, peak.max()))
multi_class_segmenter = ImageSegmenter(fib[0], classes=N_ROI, mask_alpha=0.76, ax=ax[0], props={"color":"black"})

display(widgets.HBox([slice_selector, erasing_button, class_selector, refresh_btn]))


Export the values at the current slice for each ROI into a dataframe.

In [ ]:
import pandas as pd

d = fib.shape[1] // peak.shape[1]
lbl = multi_class_segmenter.mask[::d, ::d].astype(int)
df = pd.DataFrame(
    {
        f'ROI{c}': peak[slice_selector.value, lbl == c, :].mean(axis=0)
        for c in np.unique(lbl)
    }
)
df['mass'] = mass
df['slice'] = slice_selector.value
df